In [ ]:
%pip install nilearn nibabel pandas matplotlib
%pip install --upgrade nilearn

import os
import numpy as np
import pandas as pd
import nilearn
import glob
import re
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn.image import load_img
from nilearn.plotting import plot_epi, plot_design_matrix, plot_contrast_matrix, plot_stat_map
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.contrasts import compute_contrast
from nilearn.glm.second_level import SecondLevelModel
from nilearn.plotting import plot_stat_map
from nilearn.reporting import get_clusters_table
from nilearn.image import coord_transform
from nilearn.image import resample_to_img, load_img
from nilearn.input_data import NiftiLabelsMasker

In [ ]:
print(nilearn.__version__)

### Download/load brain atlas (for anatomical labelling)

In [ ]:
from nilearn.datasets import fetch_atlas_harvard_oxford
# Fetch the Harvard-Oxford atlas
atlas = fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
atlas_img = load_img(atlas.maps)
atlas_labels = list(atlas.labels)

In [ ]:
# 1. Paths
func_dir = '/Users/onjolikrywiak/Desktop/BrainHack/sub-001/func'
reg_dir = '/Users/onjolikrywiak/Desktop/BrainHack/events/sub-001/regressors'

In [ ]:
# 2. Grab only the Lin2009cAsym bold files
pattern   = os.path.join(func_dir, '*MNI152NLin2009cAsym*_bold.nii*')
nii_files = sorted(glob.glob(pattern))
print("Found bold files:", nii_files)

In [ ]:
# 3. Loop & pair each with its confounds
for nii_path in nii_files:
    basename = os.path.basename(nii_path)
    
    # Extract subj_task and run number, e.g. "sub-001_task-empathy" + "run-01"
    m = re.match(r'(sub-[^_]+_task-[^_]+).*?(run-\d+)', basename)
    if not m:
        raise ValueError(f"Can't parse subject/task/run from {basename}")
    subj_task = m.group(1)
    run       = m.group(2)
    
    # Build confounds filename: sub-001_task-empathy_run-01_confounds.txt
    conf_name = f"{subj_task}_{run}_confounds.txt"
    conf_path = os.path.join(reg_dir, conf_name)
    if not os.path.exists(conf_path):
        raise FileNotFoundError(f"Confounds file not found: {conf_path}")
    
    # Load the bold image
    img  = nib.load(nii_path)
    data = img.get_fdata()
    print(f"{basename} → image shape {data.shape}")
    
    # Load the confounds
    # adjust sep if whitespace-delimited; here we assume tab-delimited
    conf_df = pd.read_csv(conf_path, sep='\t', header=None)
    print(f"{conf_name} → confounds shape {conf_df.shape}")
    
    # Assign to run-specific variables
    if run == 'run-01':
        img1, data1, path1, conf1 = img, data, nii_path, conf_df
    elif run == 'run-02':
        img2, data2, path2, conf2 = img, data, nii_path, conf_df

### Trim 3 timepoints from the beginning and end of BOLD

In [ ]:
# Trim 3 timepoints from the beginning and end of BOLD
trim = 3
if data.shape[-1] > 2 * trim:
    data = data[..., trim:-trim]
    print(f"Trimmed BOLD shape: {data.shape}")
else:
    raise ValueError("Not enough timepoints to trim 3 from each end.")

### Check to see if BOLD and confounds are the same length

In [ ]:
print(f"BOLD timepoints: {data.shape[-1]}")
print(f"Confounds timepoints: {conf_df.shape[0]}")

if data.shape[-1] == conf_df.shape[0]:
    print("BOLD and confounds are aligned!")
else:
    print("Warning: BOLD and confounds are NOT aligned!")

# First Level Analysis (GLM)

## Generate a Z-Map for One Trial Type (Single Subject)

In [ ]:
# 1. Parse your custom events file
events_txt = '/Users/onjolikrywiak/Desktop/BrainHack/events/sub-001/regressors/sub-001_task-empathy_timing-selfpain.txt'
with open(events_txt) as f:
    line = f.readline().strip()
pairs = line.split('\t')

# Build events DataFrame
onsets = []
durations = []
for pair in pairs:
    onset, duration = pair.split(':')
    onsets.append(float(onset))
    durations.append(float(duration))
events = pd.DataFrame({
    'onset': onsets,
    'duration': durations,
    'trial_type': 'selfpain'  # or another label if needed
})

# 2. Fit the GLM (no confounds)
fmri_glm = FirstLevelModel(
    t_r=2.0,  # set your TR
    noise_model='ar1',
    standardize=True,
    hrf_model='spm',
    drift_model='cosine'
)
fmri_glm = fmri_glm.fit(img, events=events)

# 3. Inspect design matrix columns
print("Design matrix columns:", fmri_glm.design_matrices_[0].columns.tolist())

# 4. Define and compute the contrast
contrast_name = 'selfpain'
if contrast_name in fmri_glm.design_matrices_[0].columns:
    contrast_vec = (fmri_glm.design_matrices_[0].columns == contrast_name).astype(int)
    z_map = fmri_glm.compute_contrast(contrast_vec, output_type='z_score')

    # 5. Plot the result
    plot_stat_map(z_map, title=f'{contrast_name} > baseline', threshold=3.1)
else:
    print(f"Contrast '{contrast_name}' not found in design matrix columns.")

## Generate Z-Maps for All Trial Types (Single Subject)

In [ ]:
# Path to your regressors folder
reg_dir = '/Users/onjolikrywiak/Desktop/BrainHack/events/sub-001/regressors'

# Find all timing files in the folder
event_files = glob.glob(os.path.join(reg_dir, 'sub-001_task-empathy_timing-*.txt'))

for events_txt in event_files:
    # Extract trial type from filename
    basename = os.path.basename(events_txt)
    trial_type = basename.split('timing-')[1].replace('.txt', '')

    # Parse the custom events file
    with open(events_txt) as f:
        line = f.readline().strip()
    pairs = line.split('\t')

    # Build events DataFrame
    onsets = []
    durations = []
    for pair in pairs:
        onset, duration = pair.split(':')
        onsets.append(float(onset))
        durations.append(float(duration))
    events = pd.DataFrame({
        'onset': onsets,
        'duration': durations,
        'trial_type': trial_type
    })

    # Fit the GLM (no confounds)
    fmri_glm = FirstLevelModel(
        t_r=2.0,
        noise_model='ar1',
        standardize=True,
        hrf_model='spm',
        drift_model='cosine'
    )
    fmri_glm = fmri_glm.fit(img, events=events)

    # Inspect design matrix columns
    print(f"Design matrix columns for {trial_type}:", fmri_glm.design_matrices_[0].columns.tolist())

    # Define and compute the contrast
    contrast_name = trial_type
    if contrast_name in fmri_glm.design_matrices_[0].columns:
        contrast_vec = (fmri_glm.design_matrices_[0].columns == contrast_name).astype(int)
        z_map = fmri_glm.compute_contrast(contrast_vec, output_type='z_score')

        # Plot the result
        plot_stat_map(z_map, title=f'{contrast_name} > baseline', threshold=3.1)
    else:
        print(f"Contrast '{contrast_name}' not found in design matrix columns.")

## Generate Z-Maps and Cluster Tables for All Subjects and Trial Types

In [ ]:
trial_types = ["otherneutralanticipation",
               "otherneutralcue",
               "othernopain",
               "othernopainrest",
               "otherpain",
               "otherpainanticipation",
               "otherpainpaincue",
               "otherpainrest",
               "selfneutralanticipation",
               "selfneutralcue",
               "selfnopain",
               "selfnopainrest",
               "selfpain",
               "selfpainpaincue",
               "selfpainanticipation",
               "selfpainrest"]

In [ ]:
output_dir = '/Users/onjolikrywiak/Desktop/BrainHack/outputs'
subjects = sorted({f.split('_')[0] for f in glob.glob(os.path.join(output_dir, 'sub-*_selfpain_zmap.nii.gz'))})

for subj in subjects:
    for trial_type in trial_types:
        zmap_path = os.path.join(output_dir, f"{subj}_{trial_type}_zmap.nii.gz")
        # If z-map does not exist, try to create it
        if not os.path.exists(zmap_path):
            # Try to find the functional image and events file
            func_pattern = f"/Users/onjolikrywiak/Desktop/BrainHack/{subj}/func/{subj}_*MNI152NLin2009cAsym*_bold.nii*"
            func_files = sorted(glob.glob(func_pattern))
            if not func_files:
                print(f"No functional file found for {subj}, skipping {trial_type}")
                continue
            img = load_img(func_files[0])  # You may want to refine this selection
            # Find events file
            events_pattern = f"/Users/onjolikrywiak/Desktop/BrainHack/events/{subj}/regressors/{subj}_task-empathy_timing-{trial_type}.txt"
            if not os.path.exists(events_pattern):
                print(f"No events file found for {subj} {trial_type}, skipping")
                continue
            with open(events_pattern) as f:
                line = f.readline().strip()
            pairs = line.split('\t')
            onsets, durations = [], []
            for pair in pairs:
                onset, duration = pair.split(':')
                onsets.append(float(onset))
                durations.append(float(duration))
            events = pd.DataFrame({
                'onset': onsets,
                'duration': durations,
                'trial_type': trial_type
            })
            # Fit GLM and compute z-map
            fmri_glm = FirstLevelModel(
                t_r=2.0,
                noise_model='ar1',
                standardize=True,
                hrf_model='spm',
                drift_model='cosine'
            )
            fmri_glm = fmri_glm.fit(img, events=events)
            if trial_type in fmri_glm.design_matrices_[0].columns:
                contrast_vec = (fmri_glm.design_matrices_[0].columns == trial_type).astype(int)
                z_map = fmri_glm.compute_contrast(contrast_vec, output_type='z_score')
                z_map.to_filename(zmap_path)
                print(f"Created z-map: {zmap_path}")
            else:
                print(f"Contrast {trial_type} not found for {subj}, skipping")
                continue
        else:
            z_map = load_img(zmap_path)
        # Generate cluster table
        table = get_clusters_table(z_map, stat_threshold=3.1)
        # Resample atlas to match z-map
        resampled_atlas = resample_to_img(atlas_img, z_map, interpolation='nearest')
        def get_label_from_coords(coord):
            voxel_coords = np.round(coord_transform(*coord, np.linalg.inv(resampled_atlas.affine))).astype(int)
            data = resampled_atlas.get_fdata()
            try:
                label_index = int(data[tuple(voxel_coords)])
                if label_index > 0 and label_index < len(atlas_labels):
                    return atlas_labels[label_index]
                else:
                    return 'Unknown'
            except Exception:
                return 'Unknown'
        if not table.empty and all(col in table.columns for col in ['X', 'Y', 'Z']):
            table['peak_coord'] = list(zip(table['X'], table['Y'], table['Z']))
            table['Region_Label'] = table['peak_coord'].apply(get_label_from_coords)
        else:
            table['peak_coord'] = [None] * len(table)
            table['Region_Label'] = [None] * len(table)
        # Save subject-level cluster table
        csv_path = os.path.join(output_dir, f"{subj}_{trial_type}_clusters.csv")
        table.to_csv(csv_path, index=False)
        print(f"Saved cluster table: {csv_path}")

## Summary Table for a Region of Interest (All Subjects and Trial Types)

In [ ]:
region_of_interest = "Insular Cortex"  # <-- change this to your region
output_dir = '/Users/onjolikrywiak/Desktop/BrainHack/outputs'

summary_rows = []

for csv_file in glob.glob(os.path.join(output_dir, 'sub-*_*.csv')):
    # Extract subject and trial_type from filename
    m = re.match(r'(sub-\d+)_([a-zA-Z0-9]+)_clusters\.csv', os.path.basename(csv_file))
    if not m:
        continue
    subj = m.group(1)
    trial_type = m.group(2)
    table = pd.read_csv(csv_file)
    region_rows = table[table['Region_Label'] == region_of_interest]
    for _, row in region_rows.iterrows():
        summary_rows.append({
            'Subject': subj,  # Now always just sub-001, sub-002, etc.
            'TrialType': trial_type,
            'X': row.get('X', None),
            'Y': row.get('Y', None),
            'Z': row.get('Z', None),
            'PeakStat': row.get('Peak Stat', None),
            'ClusterSize': row.get('Cluster Size (mm3)', None)
        })

summary_df = pd.DataFrame(summary_rows)
summary_df = summary_df.sort_values(by=["Subject", "TrialType"]).reset_index(drop=True)
print(summary_df)

summary_df.to_csv(os.path.join(output_dir, f"{region_of_interest.replace(' ', '_')}_summary.csv"), index=False)
print(f"Saved summary to {region_of_interest.replace(' ', '_')}_summary.csv")

# Second-level (group) analysis

## Group Z-maps and cluster tables

In [ ]:
for trial_type in trial_types:
    zmap_files = sorted(glob.glob(f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/sub-*_{trial_type}_zmap.nii.gz'))
    if not zmap_files:
        print(f"No zmaps found for {trial_type}, skipping.")
        continue
    zmap_imgs = [load_img(f) for f in zmap_files]
    print(f"Found zmaps for {trial_type}:", zmap_files)

    # Second-level analysis
    n_subs = len(zmap_imgs)
    design_matrix = pd.DataFrame([1] * n_subs, columns=['intercept'])
    second_level_model = SecondLevelModel()
    second_level_model = second_level_model.fit(zmap_imgs, design_matrix=design_matrix)
    zmap_group = second_level_model.compute_contrast('intercept', output_type='z_score')

    from nilearn.glm import threshold_stats_img
    zmap_group = second_level_model.compute_contrast('intercept', output_type='z_score')
    
    # FDR correction ## Activation is too weak to survive FDR or FWE correction
    #thresholded_map, threshold = threshold_stats_img(
    #    zmap_group, alpha=0.1, height_control='fdr'
    #    )
    #print(f"FDR threshold used: {threshold}")

    # Plot and save
    plot_stat_map(
        zmap_group,
        threshold=3.1,
        title=f'Group-level {trial_type} z-map',
        output_file=f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/group_{trial_type}_zmap.png'
    )
    zmap_group.to_filename(f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/group_{trial_type}_zmap.nii.gz')
    print(f"Saved group z-map for {trial_type}")

    # Get and save clusters table
    table = get_clusters_table(zmap_group, stat_threshold=3.1)
    
    # Resample atlas to match your group z-map
    resampled_atlas = resample_to_img(atlas_img, zmap_group, interpolation='nearest')
    
    def get_label_from_coords(coord):
        from nilearn.image import coord_transform
        import numpy as np
        # Convert MNI coordinates to voxel indices
        voxel_coords = np.round(coord_transform(*coord, np.linalg.inv(resampled_atlas.affine))).astype(int)
        data = resampled_atlas.get_fdata()
        try:
            label_index = int(data[tuple(voxel_coords)])
            if label_index > 0 and label_index < len(atlas_labels):
                return atlas_labels[label_index]
            else:
                return 'Unknown'
        except Exception:
            return 'Unknown'
        
    # Add anatomical labels to the cluster table
    if not table.empty and all(col in table.columns for col in ['X', 'Y', 'Z']):
        table['peak_coord'] = list(zip(table['X'], table['Y'], table['Z']))
        table['Region_Label'] = table['peak_coord'].apply(get_label_from_coords)
    else:
        table['peak_coord'] = [None] * len(table)
        table['Region_Label'] = [None] * len(table)

    # Save table to CSV
    csv_path = f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/group_{trial_type}_clusters.csv'
    table.to_csv(csv_path, index=False)
    print(f"Saved clusters table for {trial_type} to {csv_path}")

## How many times do anatomical regions appear across all trial types?

In [ ]:
import pandas as pd
import glob
from collections import Counter

summary = {}

# Loop through all cluster tables
for trial_type in trial_types:
    csv_path = f'/Users/onjolikrywiak/Desktop/BrainHack/outputs/group_{trial_type}_clusters.csv'
    try:
        table = pd.read_csv(csv_path)
        # Count occurrences of each region label
        region_counts = Counter(table['Region_Label'].dropna())
        summary[trial_type] = dict(region_counts)
    except Exception as e:
        print(f"Could not process {csv_path}: {e}")

# Print summary for each trial type
for trial_type, regions in summary.items():
    print(f"\n{trial_type}:")
    for region, count in regions.items():
        print(f"  {region}: {count} clusters")

# Optionally, save the summary to a CSV
summary_df = pd.DataFrame(summary).fillna(0).astype(int)
summary_df.to_csv('/Users/onjolikrywiak/Desktop/BrainHack/outputs/region_summary.csv')
print("Saved region summary to outputs/region_summary.csv")